# Snorkel Tutorial

### You will work with Wikipedia plot descriptions of films that are either comedy or drama.

First, import necessary libraries:

In [3]:
from data.preparer import load_film_dataset
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import PandasLFApplier
from analyzer import train_model
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)

## The Data

These plot descriptions are from [Kaggle](https://www.kaggle.com/jrobischon/wikipedia-movie-plots).
You will be labeling films as either "comedy" or "drama" based on their plot descriptions.

Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

In [ ]:
!unzip -P PASSWORD data/data.zip

In [5]:
df_train, df_dev, df_valid, df_test = load_film_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_test)))

8343 training examples
500 development examples
500 validation examples
1000 test examples


Define the labels for this task:

In [6]:
ABSTAIN = -1
DRAMA = 0
COMEDY = 1

In [7]:
print("Some labeled examples: ")
display(df_dev[df_dev.label==DRAMA].sample(5))
display(df_dev[df_dev.label==COMEDY].sample(5))

Some labeled examples: 


,text,label,Genre,Title
1925,"Ocean Heaven is about a terminally ill father, Sam Wong/Wang Xincheng (Jet Li) as he works his job in an aquarium and struggles to look after his 21-year-old son Dafu/David, who has autism (Wen Zhang). Sam has single-handedly brought up his son since his wife died in a swimming accident 14 years ago and looks after him day and night. Upon discovering Sam has less than five months to live, he multiplies his efforts to help Dafu learn basic tasks in order to care for himself, as well as searching for a home for him, before he passes away. As the story progresses, the growing relationship between father and son, as well as the community around them, become clearer. Ling ling (Gwei Lun-mei) plays the role of a clown who is part of a small traveling circus (they perform in the aquarium for a short period of time) and who gets on well with Dafu, leading to a close friendship between them.\r\nThough Dafu struggles to learn many basic tasks, Sam perseveres in teaching his son the tasks that he will need to know how to perform himself once his father dies. Though Sam never gave up teaching and explaining things to Dafu in a positive and supportive and memorable manner, he did contemplate ending his and Dafu's existence together rather than leave his child to suffer without him. Sam, at this point had yet to tell anyone how far his health had deteriorated, until one day while returning from their hometown, his neighbor tells Sam his doctor came from the city with bags of free medicine and waited two hours for him. Meanwhile, Principal Liu, of the boarding school for mentally challenged children that Dafu attended as a child, recommends Dafu to a newly opened institution where he can stay after his father dies.\r\nDafu continues learning, though Sam must move into the institution with him to help him adjust. Ling Ling, knowing that she is about to have to leave with her circus, teaches Dafu to pick up the phone in the aquarium when he hears it, and she will talk to him.\r\nLing Ling leaves with her circus. Sam, for the first time in the movie, swims in the aquarium with his son, dressed in a sea turtle costume he made by hand, telling his son not to be afraid when he is gone; he will be like the turtles in the aquarium, always swimming close by with him. He almost drowns but is helped from water by Mr. Tang and while they chat, Sam reveals his wife's death might not have been an accident as she was a very good swimmer and they had recently found out Dafu is autistic. Sam doesn't blame her for not being able to handle their son's circumstances. There was only one instance in which Sam loses his composure with Dafu, while he is nearing the end, and still working and trying to teach his son how to do his job, Dafu is distracted and Sam yells at him. Less than a moment later and Sam is already mending fences and reassuring his son.\r\nSam eventually dies, and his funeral is attended by close friends and family. Dafu waves good bye to the sky/Sam, as it seems he has learned people live there, something Ling Ling explained to him earlier when she spoke of her grandmother with him before she left town. After the funeral, Dafu is seen doing some of the things that his father struggled so much to teach him with success. He cooks by himself, knows how to ride the bus, and picks up the phone when Ling Ling calls, and working at the ocean park, a place Mr. has opened the doors for Dafu at any time of the day. It is implied that from then on, Dafu will know how to lead his life on his own, though he still has Sam's old friends helping him along the way.",0,drama,Ocean Heaven
1772,"Terry Scott (Ginger Rogers), who is separated from her husband, and unhappily married David Campbell (Dennis Morgan), the father of two children, meet when they are selected to serve on the jury of the Los Angeles trial of Ernest Craig (Ford Rainey). The defendant is charged with murdering his wife when she refused to grant him a divorce. While

text  \
1822  American video game designer Roger Dearly (Daniels) lives with his pet Dalmatian Pongo in London. One day, while waiting for Roger to come from a meeting with Herbert, a young prodigy who rates video games, Pongo sets his eyes on a beautiful female dalmatian named Perdy. After a frantic chase through the streets of London that starts in Soho Square Gardens and ends in St. James' Park, Roger discovers that Pongo likes Perdy. Her owner, House of De Vil fashion designer Anita Campbell-Green (Richardson), falls in love with Roger when they meet. They both fall into the pond as a result of their dogs chasing each other, but they return to Roger's home and Anita accepts his proposal. They get married at St. Luke's Church, Chelsea along with Perdy and Pongo. Meanwhile, Anita's boss, the pampered and very glamorous Cruella de Vil (Close), has a deep passion for fur, going so far as to have a taxidermist, Mr. Skinner (John Shrapnel), skin Soo-Ling, a white tiger at the London Zoo, to make it into a rug for her. Anita, inspired by her Dalmatian, designs a coat made with spotted fur. Cruella is intrigued by the idea of making garments out of actual Dalmatians and finds it amusing that it would seem as if she was wearing Anita's dog. Anita soon discovers that Perdy is pregnant and is then informed by Nanny (Plowright)that she (Anita) is, too, much to her shock. Sometime later, Cruella visits their home and expresses contempt upon meeting Roger. Her initial disgust at them having a baby turns to excitement when she finds out Perdy is expecting too. Several weeks later, she returns when a litter of 15 puppies are born and offers Roger and Anita £7,500 for them, but they refuse. Enraged, Cruella dismisses Anita and vows revenge against her, Roger and Nanny. One winter evening, she has her henchmen, Jasper and Horace (Laurie and Williams, respectively) break into their home and steal the puppies, while Roger and Anita are walking in the park with Pongo and Perdy. Along with 84 other dalmatians that were previously stolen, they deliver them to her ancient country estate, De Vil Mansion. Jasper and Horace also ask Skinner to kill and skin them to create her coat. With the family devastated at the loss of their puppies, Pongo uses the twilight bark to carry the message via the dogs and other animals of London, while Roger and Anita notify the Metropolitan Police. A dog who had witnessed the stolen puppies follows Jasper and Horace to the mansion and finds all of them inside, before helping them escape under the duo's noses. They make their way to a nearby farm, where they are later joined by Pongo and Perdy. Cruella arrives at the mansion and soon discovers what has happened. Angry with the thieves' failure, she decides to carry out the job herself, while Jasper and Horace attempt to search for them also. After several mishaps, Jasper and Horace discover nearby police on the hunt for Cruella and her henchmen and hand themselves in, joining Skinner who was beaten earlier while trying to kill Lucky, one of the 15 puppies who had been left behind. Meanwhile, Cruella tracks the puppies to the farm where they are hiding and tries to retrieve them. However, the animals outwit her, cause her to fall into a vat of molasses and get thrown through a window into a pig pen. Shortly afterwards, the fleeing dalmatians (including Lucky) are found and sent home via the Suffolk Constabulary, while those looking for Cruella arrive at the farm to arrest her. In the police van, she belittles Jasper, Horace and Skinner for their incompetence before they are sprayed by a skunk which she had mistaken for her bag. Pongo, Perdy and their puppies are reunited with Roger and Anita. After being informed that the remaining 84 puppies have no home to go to, as they have not yet been claimed by their original owners, they decide to adopt them, bringing the total to 101. Roger designs a successful video game featuring dalmatian puppies as the protagonists and Cru

## Writing Labeling Functions
Time to write some labeling functions! Below is an example. Be sure to add your function to the list `lfs`

In [70]:
lfs = []

In [50]:
@labeling_function()
def lf2(x):
    return DRAMA if "dying" in x.text.lower() else ABSTAIN

lfs.append(lf2)

## Applying Functions
This is how we obtain training labels, by training a model to combine the outputs of the noisy labeling functions.
`L_train` and `L_dev` are matrices representing the label returned by each labeling function for each example in the training and development sets.

In [73]:
# Apply the LFs to the unlabeled training data, and the development data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

100%|██████████| 500/500 [00:00<00:00, 23342.67it/s]


Train the snorkel model to combine these noisy labels.

In [74]:
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["label"] = label_model.predict(L=L_train, tie_break_policy="abstain")

## View Unlabeled Examples
You can use these to brainstorm new labeling functions. You may try filtering or sorting them in other ways.

In [79]:
# You can filter for unlabeled data
df_unlabeled = df_train[df_train.label == ABSTAIN]
display(df_unlabeled.sample(5))

text  \
7968   Abner Peacock's beloved bird-watcher's magazine, The Peacock, is in financial crisis. Desperate to stay afloat, Abner takes on a new partner, Osborn Tremain (Edmond O'Brien), who has an agenda of his own: to publish a sexy gentleman's magazine. This he and his wife Evelyn (Maureen Arthur) can only do by taking over Abner's, since he himself has been convicted too often of sending obscene material through the mails.\r\nBefore the hapless bird-watcher can stop the Tremains, the first issue sells over 40 million copies and Abner becomes the unwilling spokesman for First Amendment rights. Swept up in adulation, the unwitting playboy quickly begins settling into the swinging bachelor lifestyle.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## Analyze Results
Evaluate the accuracy of the estimated training labels and development set labels (based on ground truth).

In [75]:
Y_train = df_train.label.values
train_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(Y=Y_train)
display("Training set results:", train_analysis)

'Training set results:'

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
my_first_labeling_function,0,[0],0.109673,0.0,0.0,915,0,1.0


In [76]:
Y_dev = df_dev.label.values
dev_analysis = LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)
display("Dev set results:", dev_analysis)

'Dev set results:'

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
my_first_labeling_function,0,[0],0.124,0.0,0.0,40,22,0.645161


## Train Model
Lastly, train a simple bag of words model on these labels, and report test accuracy.

(This step may take a while).

In [ ]:
train_model(label_model, df_train, df_valid, df_test, L_train)